In [1]:
import sys, os
sys.path.append(os.path.abspath(".."))  # go up one folder

import requests, json
from IPython.display import display, Markdown, clear_output
from ollama.prompt import answer_this_prompt, generate_chat
from bn_helpers.get_structures_print_tools import get_nets, printNet, get_BN_structure, get_BN_node_states
from bn_helpers.bn_helpers import AnswerStructure, BnHelper
from bn_helpers.utils import get_path

MODEL_QUIZ = "qwen2.5:7b"
MODEL_TOOLS = "gpt-oss:latest"
# MODEL_TOOLS = MODEL_QUIZ
# print(generate_chat("Print [A, C] with no additional text", model="qwen2.5:3b", num_predict=5))
# print(answer_this_prompt("Print [A, C] with no additional text", model="qwen2.5:7b", format=AnswerStructure.model_json_schema()))

Loading Netica


In [2]:
# tool_enforced_ollama.py
import requests, json, inspect, typing

OLLAMA_URL = "http://localhost:11434/api/chat"

# ---------- Python type -> JSON Schema ----------
def _pytype_to_schema(t):
    origin = typing.get_origin(t)
    args = typing.get_args(t)
    if t in (int,): return {"type": "integer"}
    if t in (float,): return {"type": "number"}
    if t in (bool,): return {"type": "boolean"}
    if t in (str,): return {"type": "string"}
    if t in (dict, typing.Dict, typing.Mapping): return {"type": "object"}
    if t in (list, typing.List, typing.Sequence): return {"type": "array"}
    if origin is typing.Union and len(args) == 2 and type(None) in args:
        other = args[0] if args[1] is type(None) else args[1]
        sch = _pytype_to_schema(other)
        if "type" in sch and isinstance(sch["type"], str):
            sch["type"] = [sch["type"], "null"]
        return sch
    if origin in (list, typing.List, typing.Sequence) and args:
        return {"type": "array", "items": _pytype_to_schema(args[0])}
    return {"type": "string"}

def function_to_tool_schema(fn, *, name=None, description=None):
    sig = inspect.signature(fn)
    props, required = {}, []
    for pname, p in sig.parameters.items():
        if p.kind in (inspect.Parameter.VAR_POSITIONAL, inspect.Parameter.VAR_KEYWORD):
            continue
        props[pname] = _pytype_to_schema(p.annotation) if p.annotation is not inspect._empty else {"type":"string"}
        if p.default is inspect._empty:
            required.append(pname)
    return {
        "type":"function",
        "function":{
            "name": name or fn.__name__,
            "description": description or (fn.__doc__.strip() if fn.__doc__ else f"Function {fn.__name__}"),
            "parameters":{"type":"object","properties":props,"required":required}
        }
    }

def _coerce_arg(value, param: inspect.Parameter):
    if param.annotation in (int, float, bool, str):
        try: return param.annotation(value)
        except Exception: return value
    return value

def _bind_and_call(fn, kwargs: dict):
    sig = inspect.signature(fn)
    bound = {}
    for pname, p in sig.parameters.items():
        if p.kind in (inspect.Parameter.VAR_POSITIONAL, inspect.Parameter.VAR_KEYWORD):
            continue
        if pname in kwargs:
            bound[pname] = _coerce_arg(kwargs[pname], p)
        elif p.default is not inspect._empty:
            bound[pname] = p.default
        else:
            raise TypeError(f"Missing required argument: {pname}")
    return fn(**bound)

import json, requests

def _normalize_args_for_key(args):
    """
    Turn args dict into a hashable, order-stable key.
    Handles nested dicts/lists by JSON-dumping with sorted keys.
    """
    try:
        return json.dumps(args, sort_keys=True, separators=(",", ":"))
    except TypeError:
        # As a fallback, repr (still stable enough for dedup in most cases)
        return repr(args)

def chat_with_tools(
    prompt: str,
    fns: dict,
    model: str = MODEL_TOOLS,    
    temperature: float = 0.0,
    num_predict: int = 200,
    max_rounds: int = 4,
    require_tool: bool = True,
    bn_str: str = "",
    ollama_url: str = OLLAMA_URL,
):
    tools = [function_to_tool_schema(fn, name=name) for name, fn in fns.items()]

    system_prompt = (
        "You are a tool-using Bayesian Network assistant.\n"
        "Rules:\n"
        "1) Do NOT perform calculations or external actions yourself if a tool can do it.\n"
        "2) Always call a tool when any tool could plausibly answer the user.\n"
        "3) After receiving tool results, if the return value is grammatically correct, return exactly that value;\n"
        "   otherwise fix only the grammar and return the corrected value.\n"
        "4) Do NOT verify factual correctness of the tool outputs — only grammar.\n"
        # "5) If a prior tool call failed or was rejected, do NOT repeat the same tool/argument combination.\n"
        # "6) If a tool call fails, try an alternative: adjust parameters (consistent with the prompt and provided node/state names) "
        # "   or choose a different tool.\n"
    )

    # Give the model the “catalog” of node/state names to extract from
    prompt += (
        "\n\nFrom the query above, extract the correct parameters for the tools using these nodes and states below. If the query related to multiple nodes, "
        "check for the abbreviations first (e.g., 'Tuberculosis or Cancer' can be 'TbOrCa') then use the full names:\n"
        f"{bn_str}\n"
    )

    messages = [{"role":"system","content":system_prompt}, {"role":"user","content":prompt}]
    retries_left = max_rounds

    # state to avoid repeating failed/attempted calls
    seen_calls = set()          # {(tool_name, normalized_args_json)}
    recent_errors = []          # keep last few error messages for model guidance

    while retries_left > 0:
        r = requests.post(
            ollama_url,
            json={
                "model": model,
                "messages": messages,
                "tools": tools,
                "options": {"temperature": float(temperature), "num_predict": int(num_predict)},
            },
            stream=True,
        )
        if r.status_code != 200:
            raise RuntimeError(f"Ollama error {r.status_code}: {r.text}")

        assistant_msg = {"role":"assistant","content":"", "tool_calls":[]}
        for line in r.iter_lines(decode_unicode=True):
            if not line:
                continue
            try:
                chunk = json.loads(line)
            except json.JSONDecodeError:
                continue
            if "message" in chunk:
                m = chunk["message"]
                if m.get("content"):
                    assistant_msg["content"] += m["content"]
                if "tool_calls" in m and m["tool_calls"]:
                    assistant_msg["tool_calls"] = m["tool_calls"]
            if chunk.get("done"):
                break

        messages.append(assistant_msg)

        # === If the assistant requested tool calls, try to execute them ===
        if assistant_msg["tool_calls"]:
            tool_msgs = []

            for i, call in enumerate(assistant_msg["tool_calls"], 1):
                fn_name = call["function"]["name"]
                args = call["function"].get("arguments", {}) or {}

                # Normalize for dedup
                arg_key = _normalize_args_for_key(args)
                call_key = (fn_name, arg_key)

                # Skip duplicate attempts (already tried same tool+args)
                if call_key in seen_calls:
                    # Tell the model not to repeat this combination
                    recent_errors.append(
                        f"Duplicate call blocked: {fn_name}({arg_key}) was already attempted."
                    )
                    # Synthesize a 'tool' message indicating it's blocked to keep turn structure
                    payload = {"error": "DuplicateAttempt", "detail": "This exact tool/args were already tried."}
                    tool_msgs.append({
                        "role": "tool",
                        "tool_name": fn_name,
                        "content": json.dumps(payload)
                    })
                    continue

                print(f"[BayMin] tool_call #{i}: {fn_name}({args})")
                seen_calls.add(call_key)

                # Run it
                if fn_name not in fns:
                    payload = {"error": f"ToolNotRegistered", "detail": f"Tool '{fn_name}' not registered"}
                    recent_errors.append(f"{fn_name} not registered.")
                else:
                    try:
                        out = _bind_and_call(fns[fn_name], args)
                        try:
                            json.dumps(out)  # ensure JSON-serializable
                            payload = {"result": out}
                        except TypeError:
                            payload = {"result": repr(out)}
                    except Exception as e:
                        payload = {"error": type(e).__name__, "detail": str(e)}
                        recent_errors.append(f"{fn_name} failed with {type(e).__name__}: {e}")

                print(f"[BayMin] tool_result #{i}: {payload}")
                tool_msgs.append({
                    "role": "tool",
                    "tool_name": fn_name,
                    "content": json.dumps(payload)
                })

            # Attach tool results
            messages.extend(tool_msgs)

            # If any tool produced an error, ask the model to try a different tool/params (avoiding seen_calls).
            if any(json.loads(m["content"]).get("error") for m in tool_msgs):
                tried_list = [
                    f"{name}({args})" for (name, args) in list(seen_calls)[-6:]   # show up to last 6
                ]
                err_tail = "\n".join(recent_errors[-6:]) if recent_errors else "N/A"
                messages.append({
                    "role": "user",
                    "content":
                        "Some tool calls failed or were blocked. "
                        "Do NOT repeat any of the tried combinations. "
                        "Try adjusting parameters (consistent with the provided nodes/states) or choose a different tool. "
                        "Only call tools that are strictly necessary.\n"
                        f"Already tried: {tried_list}\n"
                        f"Recent issues: {err_tail}\n"
                        "If you now have sufficient tool results, finalize the answer. "
                        "Otherwise, pick a new approach."
                })
            else:
                # No errors → ask the model to finalize without calling tools again unless needed
                messages.append({
                    "role": "user",
                    "content": "Use the tool results above to answer the question. "
                               "Do not call any tools again unless strictly necessary."
                })

            retries_left -= 1
            continue

        # No tool calls returned this turn
        if assistant_msg["content"].strip():
            # Model produced direct text. If tools required, nudge once more; else return it.
            if require_tool:
                messages.append({
                    "role":"user",
                    "content": (
                        "Reminder: you must use the available tools when they can plausibly answer. "
                        "Do not answer directly. Extract parameters from the nodes/states list provided and try again. "
                        "Avoid repeating any previously tried tool/argument combinations."
                    ),
                })
                retries_left -= 1
                continue
            else:
                return assistant_msg["content"].strip()

        # If we asked to finalize but got empty, fall back to the most recent tool JSON (raw)
        if messages and any(m.get("role") == "tool" for m in messages[-5:]):
            for m in reversed(messages):
                if m.get("role") == "tool":
                    return m["content"]  # raw JSON string

        # Last resort: ask to use tools again
        messages.append({
            "role":"user",
            "content":"You returned no content. Use tools with new parameters; do not repeat prior attempts."
        })
        retries_left -= 1

    # After all rounds, return the latest assistant content if any
    for m in reversed(messages):
        if m.get("role") == "assistant" and m.get("content", "").strip():
            return m["content"].strip()
    # Or the last tool output as ultimate fallback
    for m in reversed(messages):
        if m.get("role") == "tool":
            return m["content"]
    return ""


In [3]:
nets = get_nets()
myNet = nets[1]

printNet(myNet)
print(get_BN_node_states(myNet))

# for i, net in enumerate(nets):
#   print(f"Net {i+1}:")
#   printNet(net)
#   print(get_BN_node_states(net))
#   print()


VisitAsia -> ['Tuberculosis']
Tuberculosis -> ['TbOrCa']
Smoking -> ['Cancer', 'Bronchitis']
Cancer -> ['TbOrCa']
TbOrCa -> ['XRay', 'Dyspnea']
XRay -> []
Bronchitis -> ['Dyspnea']
Dyspnea -> []
VisitAsia ['visit', 'no_visit']
Tuberculosis ['present', 'absent']
Smoking ['smoker', 'non_smoker']
Cancer ['present', 'absent']
TbOrCa ['true', 'false']
XRay ['abnormal', 'normal']
Bronchitis ['present', 'absent']
Dyspnea ['present', 'absent']



In [ ]:
def make_explain_d_connected_tool(net):
    def check_d_connected(from_node: str, to_node: str) -> dict:
        """Explain whether two nodes are d-connected and why.
        d-connected means that entering evidence for one node will change the probability of the other node.
        """
        try:
            bn_helper = BnHelper()
            is_conn = bn_helper.is_XY_connected(net, from_node, to_node)
            if is_conn:
                explanation = bn_helper.get_explain_XY_dconnected(net, from_node, to_node)
            else:
                explanation = bn_helper.get_explain_XY_dseparated(net, from_node, to_node)
            return explanation
        except Exception as e:
            return {"d_connected": None, "error": f"{type(e).__name__}: {e}"}
    return check_d_connected

def make_explain_common_cause_tool(net):
    def check_common_cause(node1, node2):
        """Check if there is a common cause between two nodes."""
        try:
            bn_helper = BnHelper()
            ans = bn_helper.get_common_cause(net, node1, node2)
            if ans:
                template = f"The common cause(s) of {node1} and {node2} is/are: {', '.join(ans)}."
            else:
                template = f"There is no common cause between {node1} and {node2}."
            return template
        except Exception as e:
            return {"common_cause": None, "error": f"{type(e).__name__}: {e}"}
    return check_common_cause

def make_explain_common_effect_tool(net):
    def check_common_effect(node1, node2):
        """Check if there is a common effect between two nodes."""
        try:
            bn_helper = BnHelper()
            ans = bn_helper.get_common_effect(net, node1, node2)
            if ans:
                template = f"The common effect(s) of {node1} and {node2} is/are: {', '.join(ans)}."
            else:
                template = f"There is no common effect between {node1} and {node2}."
            return template
        except Exception as e:
            return {"common_effect": None, "error": f"{type(e).__name__}: {e}"}
    return check_common_effect

def get_prob_node_tool(net):
    def get_prob_node(node):
        """Get the probability of a node."""
        try:
            bn_helper = BnHelper()
            prob, _ = bn_helper.get_prob_X(net, node)
            return prob
        except Exception as e:
            return {"prob_node": None, "error": f"{type(e).__name__}: {e}"}
    return get_prob_node

def get_prob_node_given_one_evidence_tool(net):
    def get_prob_node_given_one_evidence(node, evidence, evidence_state):
        """Get the probability of a node given one evidence with its state.
        If the evidence_state is not provided, it means the evidence is True."""
        try:
            bn_helper = BnHelper()
            prob, _ = bn_helper.get_prob_X_given_Y(net, node, evidence, evidence_state)
            return prob
        except Exception as e:
            return {"prob_node_given_one_evidence": None, "error": f"{type(e).__name__}: {e}"}
    return get_prob_node_given_one_evidence

def get_prob_node_given_two_evidences_tool(net):
    def get_prob_node_given_two_evidences(node, evidence1, evidence1_state, evidence2, evidence2_state):
        """Get the probability of a node given two evidences with their states.
        If the evidence_state is not provided, it means the evidence is True/or pick the state that means it is observed."""
        try:
            bn_helper = BnHelper()
            prob, _ = bn_helper.get_prob_X_given_YZ(net, node, evidence1, evidence1_state, evidence2, evidence2_state)
            return prob
        except Exception as e:
            return {"prob_node_given_two_evidences": None, "error": f"{type(e).__name__}: {e}"}
    return get_prob_node_given_two_evidences

def check_one_evidence_change_relationship_between_two_nodes_tool(net):
    def check_one_evidence_change_relationship_between_two_nodes(node1, node2, evidence):
        """Check if changing the Evidence of one node will change the dependency relationship between Node1 and Node2.
        This function is used to check if the relationship between Node1 and Node2 get affected when we observe Evidence."""
        try:
            bn_helper = BnHelper()
            ans, details = bn_helper.does_Z_change_dependency_XY(net, node1, node2, evidence)
            template = ""

            if ans:
                template = (f"Yes, observing {evidence} changes the dependency between {node1} and {node2}. "
                            f"Before observing {evidence}, {node1} and {node2} were "
                            f"{'d-connected' if details['before'] else 'd-separated'}. After observing {evidence}, they are "
                            f"{'d-connected' if details['after'] else 'd-separated'}.")
            else:
                template = (f"No, observing {evidence} does not change the dependency between {node1} and {node2}. "
                            f"Before observing {evidence}, {node1} and {node2} were "
                            f"{'d-connected' if details['before'] else 'd-separated'}. After observing {evidence}, they remain "
                            f"{'d-connected' if details['after'] else 'd-separated'}.")
            return template
        except Exception as e:
            return {"check_one_evidence_change_relationship_between_two_nodes": None, "error": f"{type(e).__name__}: {e}"}
    return check_one_evidence_change_relationship_between_two_nodes

def get_evidences_block_two_nodes_tool(net):
    def get_evidences_block_two_nodes(node1, node2):
        """Get the evidences list that block the dependency/path between Node1 and Node2."""
        try:
            bn_helper = BnHelper()
            ans = bn_helper.evidences_block_XY(net, node1, node2)
            template = f"The evidences that would block the dependency between {node1} and {node2} are: {', '.join(ans)}."
            return template
        except Exception as e:
            return {"get_evidences_block_two_nodes": None, "error": f"{type(e).__name__}: {e}"}
    return get_evidences_block_two_nodes

def get_prob_node_given_any_evidence_tool(net):
    """
    Returns a closure that can compute probabilities of any node
    given arbitrary evidence.
    
    Usage:
        tool = get_prob_node_given_any_evidence_tool(net)
        result = tool("Disease", {"Test": "Positive", "Exposure": "Yes"})
    """
    bn_helper = BnHelper()

    def get_prob_node_given_any_evidence(node: str, evidence: dict = None):
        """Get probability distribution of a node given any evidence dict."""
        try:
            prob_str, _ = bn_helper.get_prob_X_given(net, node, evidence)
            return {"result": prob_str}
        except Exception as e:
            return {
                "result": None,
                "error": f"{type(e).__name__}: {e}"
            }

    return get_prob_node_given_any_evidence

tools_map = {
    "check_d_connected": make_explain_d_connected_tool(myNet),
    "check_common_cause": make_explain_common_cause_tool(myNet),
    "check_common_effect": make_explain_common_effect_tool(myNet),
    "get_prob_node": get_prob_node_tool(myNet),
    "get_prob_node_given_any_evidence": get_prob_node_given_any_evidence_tool(myNet),
    # "get_prob_node_given_one_evidence": get_prob_node_given_one_evidence_tool(myNet),
    # "get_prob_node_given_two_evidences": get_prob_node_given_two_evidences_tool(myNet),
    "check_one_evidence_change_relationship_between_two_nodes": check_one_evidence_change_relationship_between_two_nodes_tool(myNet),
    "get_evidences_block_two_nodes": get_evidences_block_two_nodes_tool(myNet),
}

question = (
    # "Is Visiting Asia change the probability of Smoking?"
    # "Is changing the evidence of A going to change the probability of B?"
    # "What is the common effect of C and B?"
    "What is the probability of XRay given Lung Cancer, Smoking and Visit Asiaaa?"
    # "What is the probability of A given B is increased and C is present?"
    # "Is the relationship between Vizit Azia and Lung Cancr get affected when we observe Tuberculosis or Cancer?"
    # "What set of evidences would block the path between B and C?"
)

bn_str = get_BN_node_states(myNet)


print("\n=== USER QUERY ===\n", question, "\n")

answer = chat_with_tools(
    prompt=question,
    fns=tools_map,
    model=MODEL_TOOLS,
    require_tool=True,    
    temperature=0.0,
    num_predict=300,
    max_rounds=4,
    bn_str=bn_str,
)

from pydantic import BaseModel
class StructureAnswer(BaseModel):
    result: str

print("\n=== FINAL ANSWER ===\n")
# validated_answer = StructureAnswer.model_validate_json(answer)
# print(validated_answer.result)
import json

def extract_text(answer: str) -> str:
    try:
        obj = json.loads(answer)
        if isinstance(obj, dict):
            if "result" in obj["result"]:
                return obj["result"]["result"]
            if "result" in obj:
                return obj["result"]
            if "error" in obj:
                return f"Error: {obj['error']}: {obj.get('detail','')}".strip()
        return answer
    except json.JSONDecodeError:
        return answer

print(extract_text(answer))


=== USER QUERY ===
 What is the probability of XRay given Lung Cancer, Smoking and Visit Asiaaa? 



[BayMin] tool_call #1: get_prob_node_given_any_evidence({'evidence': {'Cancer': 'present', 'Smoking': 'smoker', 'VisitAsia': 'visit'}, 'node': 'XRay'})
[BayMin] tool_result #1: {'result': {'result': 'P(XRay | Cancer=present, Smoking=smoker, VisitAsia=visit):\n  P(XRay=abnormal) = 0.9800\n  P(XRay=normal) = 0.0200\n\nOriginal distribution:\n  P(XRay=abnormal) = 0.9800\n  P(XRay=normal) = 0.0200\n\nConclusion:\n  No change detected — the updated beliefs are identical to the original.\n'}}

=== FINAL ANSWER ===

P(XRay | Cancer=present, Smoking=smoker, VisitAsia=visit):
  P(XRay=abnormal) = 0.9800
  P(XRay=normal) = 0.0200

Original distribution:
  P(XRay=abnormal) = 0.9800
  P(XRay=normal) = 0.0200

Conclusion:
  No change detected — the updated beliefs are identical to the original.



In [17]:
import requests, json
from pydantic import BaseModel
from IPython.display import display, Markdown, clear_output

MODEL = "gpt-oss-bn-json"
def answer_this_prompt(prompt, stream=False, model=MODEL, temperature=0, format=None):
    payload = {
        "prompt": prompt,
        "model": model,
        "temperature": temperature,
        "max_new_tokens": 50, # only when stream = False work
        "format": format
    }
    headers = {
        'Content-Type': 'application/json'
    }
    endpoint = "http://localhost:11434/api/generate"

    # Send the POST request with streaming enabled
    with requests.post(endpoint, headers=headers, json=payload, stream=True) as response:
        if response.status_code == 200:
            try:
                # Process the response incrementally
                full_response = ""
                for line in response.iter_lines(decode_unicode=True):
                    if line.strip():  # Skip empty lines
                        response_json = json.loads(line)
                        chunk = response_json.get("response", "")
                        full_response += chunk
                        
                        # Render the response as Markdown
                        if stream:
                            clear_output(wait=True)
                            display(Markdown(full_response))
                        
                return full_response
            except json.JSONDecodeError as e:
                return "Failed to parse JSON: " + str(e)
        else:
            return "Failed to retrieve response: " + str(response.status_code)

class BnHelpers(BaseModel):
    fnName: str

def add(a=5, b=6):
    print('Go to function successfully')
    return a + b

output = answer_this_prompt('output this function name: add', stream=True, format=BnHelpers.model_json_schema())

bnHelpers = BnHelpers.model_validate_json(output)
if bnHelpers.fnName == 'add':
    print(add())

{"fnName":"add"}



Go to function successfully
11


In [16]:
bn_path = "./nets/collection/"
from bni_netica.bni_netica import *
from bni_netica.bni_netica import Net

CancerNeapolitanNet = Net(bn_path+"Cancer Neapolitan.neta")
ChestClinicNet = Net(bn_path+"ChestClinic.neta")
ClassifierNet = Net(bn_path+"Classifier.neta")
CoronaryRiskNet = Net(bn_path+"Coronary Risk.neta")
FireNet = Net(bn_path+"Fire.neta")
MendelGeneticsNet = Net(bn_path+"Mendel Genetics.neta")
RatsNet = Net(bn_path+"Rats.neta")
WetGrassNet = Net(bn_path+"Wet Grass.neta")
RatsNoisyOr = Net(bn_path+"Rats_NoisyOr.dne")
Derm = Net(bn_path+"Derm 7.9 A.dne")

BN = ""
for node in FireNet.nodes():
    BN += f"{node.name()} -> {[child.name() for child in node.children()]}\n"

def isConnected(net, fromNode, toNode):
  relatedNodes = net.node(fromNode).getRelated("d_connected")
  for node in relatedNodes:
    if node.name() == toNode:
      return True
  return False


BN = ""
for node in FireNet.nodes():
    BN += f"{node.name()} -> {[child.name() for child in node.children()]}\n"

PROMPT = "Within {BN}, is {fromNode} an ancestor of {toNode}?"
fromNode = 'Alarm'
toNode = 'Fire'

PROMPT = PROMPT.format(BN=BN, fromNode=fromNode, toNode=toNode)
inputPrompt = PROMPT + 'if user ask anything related to are these two nodes connected to each other, output this function name: isConnected'
output2 = answer_this_prompt(inputPrompt, stream=True, format=BnHelpers.model_json_schema())

{"fnName":"isConnected"}


In [ ]:
questions = [
    """In this Bayesian Networks: {BN}, is {fromNode} connected to {toNode}?""",
    """In this Bayesian Networks: {BN}, is {fromNode} connected to {toNode}? What are the two nodes mentioned?""",
    "Within the Bayesian Network {BN}, does a path exist from {fromNode} to {toNode}?",
    "In the graph {BN}, can information flow from {fromNode} to {toNode}?", # top perform 
    "Are {fromNode} and {toNode} dependent in the Bayesian Network {BN}?",
    "In {BN}, is there any direct or indirect connection between {fromNode} and {toNode}?",
    "Can {fromNode} influence {toNode} in the Bayesian Network {BN}?",
    "Is {toNode} reachable from {fromNode} in the structure of {BN}?",
    "Does {BN} contain a path that links {fromNode} to {toNode}?",
    "Are there any edges—direct or through other nodes—connecting {fromNode} and {toNode} in {BN}?",
    "Is {toNode} conditionally dependent on {fromNode} in the Bayesian Network {BN}?",
    "Within {BN}, is {fromNode} an ancestor of {toNode}?"
]

In [ ]:
listOfNets = [CancerNeapolitanNet, ChestClinicNet, ClassifierNet, CoronaryRiskNet, FireNet, MendelGeneticsNet, RatsNet, WetGrassNet, RatsNoisyOr, Derm]

for question in questions:
  total = 0
  correct = 0
  print(f"Question: {question.format(BN=net.name(), fromNode=fromNode, toNode=toNode)}")
  for net in listOfNets:
      for _ in range(5):
        total += 1
        fromNode, toNode = pickTwoRandomNodes(net)
        if fromNode and toNode:
            
            correctIdentified, queryFromNode, queryToNode = correctIdentification(question, net, fromNode, toNode)
            if correctIdentified:
              correct += 1
            else:
              print(f"Incorrect identification for {net.name()}")
              printNet(net)
              print()
              print("Expected:", fromNode, "->", toNode)
              print("Reality:", queryFromNode, "->", queryToNode)
              print("----------------------------------------------------")

  print(f"Total: {total}, Correct: {correct}, Accuracy: {correct/total:.2%}")
  print("<------------------------------------------------------------------------->")

In [ ]:
from bni_netica.support_tools import get_nets, printNet, get_BN_structure, get_BN_node_states
from bni_netica.bn_helpers import BnHelper, QueryTwoNodes, ParamExtractor
from ollama.prompt import answer_this_prompt
from bni_netica.scripts import HELLO_SCRIPT, MENU_SCRIPT, GET_FN_SCRIPT

# PROMPT = """Consider this question: '{question}'. 
# What are the two nodes in this question? 
# Make sure to correctly output the names of nodes exactly as mentioned in the network and in the order as the question mentioned. 
# For example, if the question mentioned "A and B" then the two nodes are fromNode: A, toNode: B; or if the question mentioned "Smoking and Cancer" then the two nodes are fromNode: Smoking, toNode: Cancer. 
# Answer in JSON format."""

def query_menu(BN_string, net):
    """Input: BN: string, net: object"""
    pre_query = f"""In this Bayesian Network: 
{BN_string}
"""
    user_query = input("Enter your query here: ")
    get_fn_prompt = pre_query + "\n" + user_query + GET_FN_SCRIPT

    get_fn = answer_this_prompt(get_fn_prompt, format=BnHelper.model_json_schema())
    print("\nBayMin:")
    print(get_fn)

    get_fn = BnHelper.model_validate_json(get_fn)
    fn = get_fn.function_name

    bn_helper = BnHelper(function_name=fn)
    param_extractor = ParamExtractor()
    
    if fn == "is_XY_connected":
        
        get_params = param_extractor.extract_two_nodes_from_query(pre_query, user_query)
        print(get_params)

        ans = bn_helper.is_XY_connected(net, get_params.from_node, get_params.to_node)

        if ans:
            template = f"Yes, {get_params.from_node} is d-connected to {get_params.to_node}, which means that entering evidence for {get_params.from_node} would change the probability of {get_params.to_node} and vice versa."
        else:
            template = f"No, {get_params.from_node} is not d-connected to {get_params.to_node}, which means that entering evidence for {get_params.from_node} would not change the probability of {get_params.to_node}."
        
        explain_prompt = f"""User asked: In this '{BN_string}', '{user_query}'. We use {fn} function and the output is: '{ans}'. Follow this exact template to provide the answer: '{template}'."""
        print(answer_this_prompt(explain_prompt))

    print()
    
    print(MENU_SCRIPT)
    choice = int(input("Enter your choice: "))
    print()

    if choice == 1:
        input("Enter your query here: ")
        print('This is a sample answer.\n')
    elif choice == 2:
        input("Enter your query here: ")
        print('This is a sample answer.\n')
    elif choice == 3:
        print("Not yet implemented\n")
        return 
    elif choice == 4:
        print("Goodbye!\n")
        return    

def main():
    print(HELLO_SCRIPT)
    nets = get_nets()

    
    for i, net in enumerate(nets):
        print(f"{i}: {net.name()}")

    print()
    choice = int(input("Enter the number of the network you want to use: "))
    print()
    if choice < 0 or choice >= len(nets):
        print("Invalid choice. Exiting.")
        return
    
    net = nets[choice]
    print(f"You chose: {net.name()}")
    printNet(net)
    print('\nBN states:\n')
    print(get_BN_node_states(net))

    BN_string = get_BN_structure(net)
    query_menu(BN_string=BN_string, net=net)


if __name__ == "__main__":
    main()
